In [1]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline
from matplotlib.pyplot import figure
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'Malgun Gothic' 
plt.rcParams["font.size"] = 10
%matplotlib inline 


from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

from sklearn.preprocessing import StandardScaler
import os

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dataset/test.csv
/kaggle/input/dataset/train.csv
/kaggle/input/dataset/TEST_DHweightDaconArima_NAFilled.csv
/kaggle/input/dataset/submission.csv
/kaggle/input/dataset/TRAIN_DHweightDaconArima_NAFilled.csv
/kaggle/input/weather/hourly.csv
/kaggle/input/weather/daily.csv
/kaggle/input/weather/monthly.csv


## Weather 추가변수
- 나중에 모델 돌릴때마다 df에 추가하는 것

In [3]:
# hourly WEATHER data
w_hr = pd.read_csv("/kaggle/input/weather/hourly.csv",encoding='euc-kr', sep=',')
w_hr2 = w_hr.loc[:,['Time', 'Temp','Humidity', 'WindSpeed']]
w_hr2['Humidity']=w_hr2.loc[:,'Humidity'].fillna(0)
w_hr2.head(2)

,Time,Temp,Humidity,WindSpeed
0,2016.7.20 0:00,25.7,0.0,1.3
1,2016.7.20 1:00,25.6,0.0,1.4


In [4]:
w_hr2['Time'] = pd.to_datetime(w_hr2.Time)
w_hr2 = w_hr2.set_index('Time')
w_hr2.head(2)

,Temp,Humidity,WindSpeed
Time,,,
2016-07-20 00:00:00,25.7,0.0,1.3
2016-07-20 01:00:00,25.6,0.0,1.4


## 파생 및 dummy변수

In [5]:
test = pd.read_csv("/kaggle/input/dataset/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [6]:
test.index = pd.to_datetime(test.index)

In [7]:
def dateTimeVar(df):
    df1 = pd.DataFrame(df.index)
    df1.index = pd.to_datetime(df1.iloc[:,0])
    df1 = df1.drop(columns=df1.columns)
    # datetime을 사용한 파생변수
    # 연도, 월, 일, 요일, 시간
    df1['Mth']= df1.index.month
    df1['Date']=df1.index.day
    df1['Day']=df1.index.weekday
    df1['hour']=df1.index.hour
    
    # Season 1: 봄 ~ 4: 겨울 
    df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
                1 if x in [3,4,5] else 2 if x in [6,7,8]
               else 3)
    # dayClass 1: 월, 2: 화~금, 3: 토, 일 
    # 참조 from 실시간 수요예측 기법 개발 및 온라인 수요예측 방안 연구_KPX 한국전력거래소)
    df1['dayClass'] = df1.Day.apply(lambda x: 1 if x ==0 else 
                2 if x in [1,2,3,4] else 3)
    return df1

In [8]:
TE_time_vars = dateTimeVar(test)

In [9]:
TE_time_vars.head(1)

,Mth,Date,Day,hour,season,dayClass
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2


## Make DUMMIES

In [10]:
test.head(2)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667
2017-08-29 11:00:00,1.458903,0.398,0.293,0.358,0.350,0.232,0.253,0.260,0.168,0.084,...,0.163,0.086,0.288,0.342010,1.150483,0.196,0.343,1.109,0.225,0.29667


In [11]:
TE_time_vars.head(2)

,Mth,Date,Day,hour,season,dayClass
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2
2017-08-29 11:00:00,8,29,1,11,2,2


In [12]:
def dummies(df, dum_lst):
    df1 = df.copy()
    # dum_lst에 들어있는 각각의 변수마다 
    # tmp = get_dummies, 해당 original 변수는 drop
    # tmp와 변수가 drop된 df를 join 후 리턴
    for v in dum_lst:
        tmp = pd.get_dummies(df1[v], prefix=v)
        df1 = df1.drop(columns=v).join(tmp)
    return df1

In [13]:
TE_timeV_dum = dummies(TE_time_vars, ['season', 'dayClass'])

In [14]:
TE_timeV_dum_wthr= pd.concat([TE_timeV_dum, w_hr2], axis=1,join='inner')

In [15]:
TE_timeV_dum_wthr.head(2)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0
2017-08-29 11:00:00,8,29,1,11,0,1,0,0,0,1,0,19.7,0.0,4.7


# Modeling

* test : TEST셋의 순수한 meterIDs Time series (NA값 채워짐) 
* TE_timeV_dum: TEST셋의 시간파생변수 + dummy variables 

In [16]:
test.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667


In [21]:
TE_timeV_dum_wthr.head(1)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0


### 칼럼 50개 돌리기

In [22]:
##################################################
#### 칼럼 바꾸기 ###################################

test50 = test.iloc[:,:50]
test50.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X371,X200,X111,X163,X364,X378,X271,X462,X427,X73
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.214,0.303754,0.445,0.203,0.249,0.22,0.311651,0.268794,0.27,0.008


In [23]:
submission_hourly = pd.date_range(start = pd.datetime(2018, 7, 1, 0),
                             end = pd.datetime(2018, 7, 1, 23),
                             freq = "H")
tmp_test = pd.DataFrame(index = submission_hourly)
X_test_H = dummies(dateTimeVar(tmp_test), ['season', 'dayClass'])
X_test_H['season_1'] =0 ; X_test_H['season_3'] =0 ;X_test_H['season_4'] =0
X_test_H['dayClass_1'] =0 ; X_test_H['dayClass_2'] =0 
X_test_H = pd.concat([X_test_H, w_hr2], axis=1,join='inner')

In [24]:
X_test_H.head(3)

,Mth,Date,Day,hour,season_2,dayClass_3,season_1,season_3,season_4,dayClass_1,dayClass_2,Temp,Humidity,WindSpeed
2018-07-01 00:00:00,7,1,6,0,1,1,0,0,0,0,0,23.7,0.0,1.7
2018-07-01 01:00:00,7,1,6,1,1,1,0,0,0,0,0,23.7,0.0,2.0
2018-07-01 02:00:00,7,1,6,2,1,1,0,0,0,0,0,23.8,0.0,2.0


In [31]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lg_param = {"boosting_type":['gbdt', 'dart','goss'],
    "learning_rate": [0.01,0.05,0.1,0.15],
    "min_child_samples":[10, 20, 30, 50],       
    "n_estimators":[50, 100, 200, 500],
    }

def gsLGBR_Hrly_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=3, verbose=5)
    grid_gb.fit(X_train, y_train)
    fcst = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst[i]]
    return a
   
def test50_Hrly_GS_LGBReg(train, X_test_H):
    hrly_50 = pd.DataFrame()
    for mID in train.columns:
        data = pd.concat([train.loc[:,[mID]], TE_timeV_dum_wthr], axis=1)
        data = data.rename(columns={mID:'y'})
        for i in [24, 168]:
            data['lag_{}'.format(i)] = data.y.shift(i)
        y_train = data.dropna().y
        X_train = data.dropna().drop(['y'], axis=1)
        
        X_test_H['lag_24'] = y_train.tail(24).values
        X_test_H['lag_168'] = data.loc[:,'lag_168'][7166:7190].values

        tmp = gsLGBR_Hrly_FitPred(LGBMRegressor(verbose=1), lg_param,
                        mID, X_train, y_train, X_test_H)
        hrly_50 = pd.concat([hrly_50, tmp], axis=0)
    return hrly_50

In [26]:
pred_50 = test50_Hrly_GS_LGBReg(test50, X_test_H)

Fitting 3 folds for each of 192 candidates, totalling 576 fits
[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=50 
[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=50, score=0.576, total=   0.2s
[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=50 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=50, score=0.581, total=   0.1s
[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=50 
[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=50, score=0.028, total=   0.1s
[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=100 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s


[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=100, score=0.786, total=   0.3s
[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=100 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s


[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=100, score=0.817, total=   0.3s
[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=100 
[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=100, score=0.043, total=   0.3s
[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=200 
[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=200, score=0.864, total=   0.5s
[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=200 
[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=200, score=0.926, total=   0.5s
[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=200 
[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=200, score=0.052, total=   0.5s
[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=500 
[CV]  boos

[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=50, n_estimators=500, score=0.842, total=   1.2s
[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=50, n_estimators=500 


KeyboardInterrupt: 

In [ ]:
import os
os.chdir(r'kaggle/working')
pred_50.to_csv(r'first50_Hrly.csv')

from IPython.display import FileLink
FileLink(r'first50_Hrly.csv')